# PHYS6318: Lecture 8

In [180]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
from matplotlib import rcParams
rcParams['animation.html'] = 'jshtml'
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import netCDF4
import time
import os
import glob

## OMB 5.10 Exercise 14: Island Wakes

### 5.10.1 Aim
The aim of this exercise is to simulate turbulent wakes produced by horizontal flows
around an island. This includes implementations of both lateral friction and lateral
momentum diffusion in the shallow-water model.

### 5.10.2 The Reynolds Number

The ratio between the nonlinear terms and the diffusion of momentum is call the *Reynolds number* and can be defined by:

$$Re = \frac{UL}{A_h} $$

where $U$ is the speed of the incident flow, $L$ is the diameter of the obstacle, and $A_h$
is ambient horizontal eddy viscosity

For small values of $Re \approx 1$, the flow is laminar.  For larger values ($Re \gg 100$), a tubulent wake with a organized pattern of *von Karman vortices* shedding at a frequency.  We investigate that phenomenom here.

### 5.10.3 Inclusion of Lateral Friction and Momentum Diffusion

To simulate the development of turbulent wakes in the lee of a obstacle, we need lateral friction and the diffusion of momentum.  Assuming uniform lateral eddy viscosity $A_h$, the depth averaged version of the lateral momentum equation is

$$\begin{align}
\mbox{div}_h (u) &= \frac{A_h}{h} \left\{ \frac{\partial}{\partial x}\left(h \frac{\partial u }{\partial x}\right)  + \frac{\partial}{\partial y}\left(h \frac{\partial u }{\partial y}\right)\right\} \\
\mbox{div}_h (v) &= \frac{A_h}{h} \left\{ \frac{\partial}{\partial x}\left(h \frac{\partial v }{\partial x}\right)  + \frac{\partial}{\partial y}\left(h \frac{\partial v }{\partial y}\right)\right\} 
\end{align}$$

where $h$ is the layer thickness.

The discretized form of diffusion terms are given in equations (OMB 5.35 and 5.35).  See Figure 5.16 for details.

### 5.10.4 Stability  Criterion for Diffusion Terms

For a one-dimensional diffusion equation of the form

$$ \frac{\partial \psi}{\partial t} = A_h \frac{\partial^2 \psi}{\partial x^2} $$

Using an explicit finite-difference formulation leads to a stability criterion:

$$ \Delta t \le \frac{(\Delta x)^2}{A_h} $$

We can assume and upper bound on the time step for our momentum equations will be similar to this.

### 5.10.5 Full-Slip, Semi-Slip and No-Slip Conditions

#### Full-slip

No shear flow parallel to the coast $\implies$ no frictional stress imposed at the boundary.  

Implemented with $\hat{n} \cdot \nabla \vec{u} = 0$ (zero-gradient condition) along the boundary. This means that 

$$ v_{j, k+1} - v_{j,k} = 0 $$

across a coastline and gives $v_{\mbox{coast}} = v_{j, k}$.

#### Zero-slip

A zero-slip (or no-slip) condition requires at $\vec{u} = 0$ along the boundaries.  To acheive this, we set the grid point just across a boundary to be anti-parallel to the coastal flow.  This means that

$$ v_{j, k+1} = - v_{j,k} = 0 $$

across a coastline and gives $v_{\mbox{coast}} = 0 $

#### Semi-slip

An intermediate condition called semi-slip gives half the velocity shear compared to the no-slip condition.  In this case, we set the grid point just across a boundary to be zero. This means that

$v_{j, k+1} = 0$.  

across a coastline and gives $v_{\mbox{coast}} = 0.5 v_{j,k} $


See OMB Fig 5.17 for an illustration.


### 5.10.6 Task Description

Create bathymetry with a 10 km long, 5 km channel that is 10 m deep.  Near the western boundary, place a small hill in the centre of the channel.  The north and south boundaries are closed with a no-slip boundary condition. 

The west and east boundaries are open, cyclic boundaries meaning any fluid that escapes on one side an reappears on the other side.


In [3]:
nx = 101
ny = 51
dx = 100 # m
dy = 100 # m
dt = 3 # s

Force the model with a westerly (eastward) wind with $\tau^{\mbox{wind}}_x = 0.2$ Pa. This will result in a flow speed of about 0.5 m/s.

Use the TVD Superbee scheme for advection of any property.  

We will introduce a point source of an Eulerian tracer concentration at the western boundary for visualization of the flow dynamics.  Unlike the dynamical variables ($u$, $v$, $\eta$) the Eulerian tracer is allowed to dissappear through the eastern boundary using a zero-gradient condition.

### 5.10.7 Sample Code

See `src/OMB_Ex14` for version with NetCDF output.

## Visualization routine from previous lecture

#### Transport

In [156]:
def calc_transport(u, v, h):
    
    nx, ny = u.shape
    uh = np.zeros_like(u)
    vh = np.zeros_like(v)
    
    for j in range(1, ny-1): 
        for k in range(1, nx-1):
            uh[k, j] = u[k, j]*0.5*(h[k,j] + h[k+1, j])
            vh[k, j] = v[k, j]*0.5*(h[k,j] + h[k, j+1])

    return uh, vh

In [157]:
def animate_transport(filename):
    ds = xr.open_dataset(filename)
    fig, ax = plt.subplots(figsize=(8,4))

    # show time in units of days
    t = (ds.time - ds.time[0]).values / np.timedelta64(1, 'D')

    ax.contour(ds.xh/1e3, ds.yh/1e3, ds.h[0].T, colors='k', linewidths=0.5)
    
    # compute grid centres
    xu, yu = np.meshgrid(ds.xu, ds.yu)
    xv, yv = np.meshgrid(ds.xv, ds.yv)
    xc = 0.5*(xu + xv)
    yc = 0.5*(yu + yv)
    
    uh, vh = calc_transport(ds.u[0].values, ds.v[0].values, ds.h[0].values)
    
    q = ax.quiver(xc[::3, ::3]/1e3,
                  yc[::3, ::3]/1e3, 
                  uh[::3, ::3].T, 
                  vh[::3 ,::3].T, 
                  scale=70)
    text = ax.text(4, 5.3, f'time= {t[0]:.0f} days')

    def init():
        #ax.set_title('Transport')
        ax.set_xlabel('x (km)')
        ax.set_ylabel('y (km)')
        
        return q, text
    
    def update(frame):

        uh, vh = calc_transport(ds.u[frame].values, ds.v[frame].values, ds.h[frame].values)
        
        q.set_UVC(uh[::3, ::3].T, vh[::3, ::3].T)
        text.set_text(f'time= {t[frame]:.1f} days')
        
        return q, text
    
        
    anim = FuncAnimation(fig, update, blit=True,
                         init_func=init,
                         #frames=len(ds.time),
                         frames=np.arange(0, len(ds.time), 10),
                        )
    plt.close(fig)
    
    return anim


### 5.10.8 Results

#### Case 1: low Reynolds number

$$ A_h = 2.5\;\mbox{m}^2/\mbox{s} $$

With the prescribed wind, the flow is about $U = 0.5\;\mbox{m}/\mbox{s}$ and the diameter of the island is $L = 300\;\mbox{m}$.  So the Reynolds number is

$$Re = \frac{UL}{A_h} = \frac{(0.5)(300)}{2.5} = 60$$


In [190]:
filename = 'src/OMB_Ex14/output_60.nc'
anim = animate_transport(filename)

In [187]:
moviefile = 'Lecture09_mov1.mp4'
anim.save(moviefile, writer='ffmpeg', fps=5, dpi=300)

In [188]:
HTML(f'<center><video controls autoplay src="{moviefile}" width=100%/></center>')

In [216]:
def animate_flow(filename, variable='T'):
    ds = xr.open_dataset(filename)
    fig, ax = plt.subplots(figsize=(12,5))

    # show time in units of days
    t = (ds.time - ds.time[0]).values / np.timedelta64(1, 'h')

    ax.contour(ds.xh/1e3, ds.yh/1e3, ds.h[0].T, colors='k', linewidths=0.5)
    
    xh, yh = np.meshgrid(ds.xh, ds.yh)
    
    # compute grid centres
    xu, yu = np.meshgrid(ds.xu, ds.yu)
    xv, yv = np.meshgrid(ds.xv, ds.yv)
    xc = 0.5*(xu + xv)
    yc = 0.5*(yu + yv)
    
    uh, vh = calc_transport(ds.u[0].values, ds.v[0].values, ds.h[0].values)
    
    pmesh = ax.pcolormesh(xh/1e3, yh/1e3, ds[variable][0].T, 
                      vmin=0, vmax=1,
                     shading='gouraud')
    
    q = ax.quiver(xc[::3, ::3]/1e3,
                  yc[::3, ::3]/1e3, 
                  uh[::3, ::3].T, 
                  vh[::3 ,::3].T, 
                  scale=70)
    
    fig.colorbar(pmesh, ax=ax)
    
    text = ax.text(4, 5.3, f'time= {t[0]:.0f} hour(s)')

    def init():
        
        
        ax.set_xlabel('x (km)')
        ax.set_ylabel('y (km)')
        ax.set_xlim(0, 10)
        
        return q, text, pmesh
    
    def update(frame):

        pmesh.set_array(ds[variable][frame].values.T.flatten())
        uh, vh = calc_transport(ds.u[frame].values, ds.v[frame].values, ds.h[frame].values)
        
        q.set_UVC(uh[::3, ::3].T, vh[::3, ::3].T)
        text.set_text(f'time= {t[frame]:.1f} hour(s)')
        
        return q, text, pmesh
    
        
    anim = FuncAnimation(fig, update, blit=True,
                         init_func=init,
                         #frames=np.arange(0, len(ds.time), 10),
                         frames=len(ds.time),
                        )
    plt.close(fig)
    
    return anim

In [220]:
filename = 'src/OMB_Ex14/output_60.nc'
anim = animate_flow(filename)

moviefile = 'Lecture09_mov2.mp4'
anim.save(moviefile, writer='ffmpeg', fps=10, dpi=200)

HTML(f'<center><video controls autoplay src="{moviefile}" width=100%/></center>')

#### Case 2: moderate Reynolds number

$$ A_h = 1.0\;\mbox{m}^2/\mbox{s} $$

With the prescribed wind, the flow is about $U = 0.5\;\mbox{m}/\mbox{s}$ and the diameter of the island is $L = 300\;\mbox{m}$.  So the Reynolds number is

$$Re = \frac{UL}{A_h} = \frac{(0.5)(300)}{1.0} = 150$$


In [192]:
filename = 'src/OMB_Ex14/output_150.nc'
anim = animate_flow(filename)

moviefile = 'Lecture09_mov3.mp4'
anim.save(moviefile, writer='ffmpeg', fps=10, dpi=300)

HTML(f'<center><video controls autoplay src="{moviefile}" width=100%/></center>')

#### Case 3: larger Reynolds number

$$ A_h = 0.5\;\mbox{m}^2/\mbox{s} $$

With the prescribed wind, the flow is about $U = 0.5\;\mbox{m}/\mbox{s}$ and the diameter of the island is $L = 300\;\mbox{m}$.  So the Reynolds number is

$$Re = \frac{UL}{A_h} = \frac{(0.5)(300)}{0.5} = 300$$


In [193]:
filename = 'src/OMB_Ex14/output_300.nc'
anim = animate_flow(filename)

moviefile = 'Lecture09_mov4.mp4'
anim.save(moviefile, writer='ffmpeg', fps=10, dpi=300)

HTML(f'<center><video controls autoplay src="{moviefile}" width=100%/></center>')

Compare t=30 hours between Case 2 and Case 3